In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress

# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 1. HELPER FUNCTIONS (Your Credit Model)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score"""
    try:
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        elif 'Total Liabilities' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities'].iloc[0]
        else:
            return np.nan
        
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    try:
        limit = min(4, len(financials.columns))
        years = financials.columns[:limit]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        if len(margins) < 2: return "N/A"
        slope, _, _, _, _ = linregress(range(len(margins)), margins[::-1])
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"

def get_interest_coverage(financials):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        if 'Interest Expense' in financials.index: interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index: interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else: return 100.0
        
        interest = abs(interest)
        if interest == 0: return 100.0
        return round(ebit / interest, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        tax_prov = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        tax_rate = max(0.0, min(tax_prov / pre_tax, 0.30)) if pre_tax > 0 else 0.21
        nopat = ebit * (1 - tax_rate)

        equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        debt = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else 0
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested = equity + debt - cash
        if invested <= 0: return np.nan
        return round((nopat / invested) * 100, 2)
    except:
        return np.nan

# ==========================================
# 2. STEP 1: FETCH UNIVERSE (US + Canada)
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    
    # 1. USA (NASDAQ Traded List - Huge list)
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        # Filter junk
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        
        # Clean list: Remove warrants/preferreds (basic string check)
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")

    # 2. CANADA (TSX Composite from Wikipedia)
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        # Mimic browser to avoid block
        headers = {"User-Agent": "Mozilla/5.0"} 
        r = requests.get(url_ca, headers=headers)
        df_ca = pd.read_html(io.StringIO(r.text))[0]
        
        # TSX tickers need .TO suffix
        if 'Symbol' in df_ca.columns:
            ca_list = df_ca['Symbol'].apply(lambda x: x.replace('.', '-') + ".TO").tolist()
            tickers.extend(ca_list)
            print(f"   -> Found {len(ca_list)} Canadian candidates.")
    except Exception as e:
        print(f"   Error fetching Canada list: {e}")

    return list(set(tickers))

# ==========================================
# 3. STEP 2: SPLIT FILTER (The Magic Part)
# ==========================================
def filter_universe_split(ticker_list):
    print(f"\n--- STEP 2: Applying Split Filters (YahooQuery) ---")
    print("   US Criteria:     Price > $5, Cap > $300M")
    print("   Canada Criteria: Price > $5, Cap > $100M, Vol > 100k")
    
    valid_candidates = []
    
    # Batch size for YahooQuery
    chunk_size = 500
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {i+len(chunk)}...", end='\r')
        
        try:
            # Use YahooQuery to get Summary Detail (contains Cap, Price, Vol)
            yq = Ticker(chunk, asynchronous=True)
            data = yq.summary_detail
            price_data = yq.price
            
            for symbol in chunk:
                # Handle Data Availability
                if symbol not in data or isinstance(data[symbol], str): continue
                if symbol not in price_data or isinstance(price_data[symbol], str): continue
                
                # Extract Metrics
                details = data[symbol]
                prices = price_data[symbol]
                
                mkt_cap = details.get('marketCap', 0)
                # Volume: use 'averageVolume' or 'volume'
                avg_vol = details.get('averageVolume', 0)
                # Price: check price dict
                curr_price = prices.get('regularMarketPrice', 0)
                
                if mkt_cap is None: mkt_cap = 0
                if avg_vol is None: avg_vol = 0
                if curr_price is None: curr_price = 0

                # --- SPLIT LOGIC ---
                is_canada = symbol.endswith('.TO')
                
                if is_canada:
                    # Canada Rules: Price > 5, Vol > 100k, Cap > 100M
                    if (curr_price > 5.0) and (avg_vol > 100000) and (mkt_cap > 100_000_000):
                        valid_candidates.append({
                            'Ticker': symbol, 
                            'Price': curr_price,
                            'Market_Cap': mkt_cap,
                            'Region': 'Canada'
                        })
                else:
                    # US Rules: Price > 5, Cap > 300M (Volume not specified, but good to ensure liquidity)
                    if (curr_price > 5.0) and (mkt_cap > 300_000_000) and (avg_vol > 1000000):
                        valid_candidates.append({
                            'Ticker': symbol, 
                            'Price': curr_price,
                            'Market_Cap': mkt_cap,
                            'Region': 'USA'
                        })
                        
        except Exception as e:
            # print(f"Batch Error: {e}")
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

# ==========================================
# 4. STEP 3: DEEP DIVE (Credit Model)
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: return
    
    print(f"\n--- STEP 3: Running Credit Model on {len(candidates_df)} Stocks ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.1) # Tiny sleep to be polite
            
        try:
            stock = yf.Ticker(ticker)
            
            # Fetch Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            
            # Skip Financials
            if 'Financial' in info.get('sector', ''): continue
            
            # Run Math
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Price': row['Price'],
                'Region': row['Region'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic
            }
            
            # Buckets (Your Criteria)
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

# --- MAIN EXECUTION ---
# 1. Get Universe
raw_tickers = get_raw_universe()

# 2. Split Filter (US vs Canada)
filtered_df = filter_universe_split(raw_tickers)

# 3. Run Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Picks) ---")
        display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    
    if not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS ---")
        display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Applying Split Filters (YahooQuery) ---
   US Criteria:     Price > $5, Cap > $300M
   Canada Criteria: Price > $5, Cap > $100M, Vol > 100k
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 1534

--- STEP 3: Running Credit Model on 1534 Stocks ---
   Analyzing 1530/1534...

--- FORTRESS STOCKS (Top Picks) ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC
97,WPM,124.22,USA,206.55,Improving,2569.34,9.30
131,CRDO,144.83,USA,124.39,Improving,100.00,7.83
102,NVDA,190.53,USA,91.34,Improving,329.77,87.21
37,ARM,110.27,USA,35.37,Stable,100.00,16.26
174,AVAV,255.00,USA,34.35,Improving,27.04,6.36
29,APP,714.23,USA,32.58,Improving,5.89,47.98
194,FAST,41.56,USA,30.85,Stable,206.85,29.96
5,DOCS,43.69,USA,29.31,Improving,100.00,22.01
73,MNST,77.31,USA,29.27,Stable,100.00,30.51
28,RMBS,94.11,USA,28.56,Improving,126.44,15.31



--- MOONSHOT STOCKS ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC
36,CAKE,52.55,USA,2.48,Improving,21.95,8.97
47,MBC,11.27,USA,2.47,Improving,3.37,8.28
7,EGO,37.18,USA,2.47,Improving,20.69,7.33
14,OC,113.74,USA,2.43,Improving,8.02,11.52
1,TRGP,182.90,USA,2.43,Improving,3.51,12.94
29,JBI,6.77,USA,2.42,Improving,3.20,11.51
27,HAYW,15.94,USA,2.42,Improving,3.16,8.04
42,HBM,20.22,USA,2.40,Improving,4.17,9.26
50,RDY,14.16,USA,2.38,Improving,25.46,14.69
43,APTV,76.91,USA,2.38,Improving,6.04,11.32
